In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))
x = x/255

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_x_letter = x_letter[-valid_size:]
x_letter = x_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(x_letter.shape)
print(valid_x_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6000, 26)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(4800, 28, 28, 1)
(1200, 28, 28, 1)
(4800, 10)
(1200, 10)
(4800, 26)
(1200, 26)


In [10]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    
    return model

In [11]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48],
                   'myDenseNet':[8, 12, 16, 32]}

base_growth_rate = 32

model_input = Input(shape=(28,28,1))
classes = 10

model = DenseNet(model_input, classes, 'DenseNet-121')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [12]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4800 samples, validate on 1200 samples
Epoch 1/100
4800/4800 [==============================] - 57s 12ms/step - loss: 2.3231 - acc: 0.2544 - val_loss: 2.4634 - val_acc: 0.0975
Epoch 2/100
4800/4800 [==============================] - 30s 6ms/step - loss: 1.2727 - acc: 0.5548 - val_loss: 2.2533 - val_acc: 0.2083
Epoch 3/100
4800/4800 [==============================] - 29s 6ms/step - loss: 0.9258 - acc: 0.6785 - val_loss: 3.7176 - val_acc: 0.1850
Epoch 4/100
4800/4800 [==============================] - 30s 6ms/step - loss: 0.7002 - acc: 0.7615 - val_loss: 2.7710 - val_acc: 0.3583
Epoch 5/100
4800/4800 [==============================] - 29s 6ms/step - loss: 0.5342 - acc: 0.8219 - val_loss: 2.0168 - val_acc: 0.4833
Epoch 6/100
4800/4800 [==============================] - 30s 6ms/step - loss: 0.4737 - acc: 0.8429 - val_loss: 1.2177 - val_acc: 0.6608
Epoch 7/100
4800/4800 [==============================] - 30s 6ms/step - loss: 0.3802 - acc: 0.8742 - val_loss: 1.1870 - val_acc: 0.6942

4800/4800 [==============================] - 30s 6ms/step - loss: 2.8915e-05 - acc: 1.0000 - val_loss: 0.1631 - val_acc: 0.9550
Epoch 61/100
4800/4800 [==============================] - 30s 6ms/step - loss: 3.6686e-05 - acc: 1.0000 - val_loss: 0.1616 - val_acc: 0.9558
Epoch 62/100
4800/4800 [==============================] - 30s 6ms/step - loss: 2.9365e-05 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9558
Epoch 63/100
4800/4800 [==============================] - 30s 6ms/step - loss: 3.1600e-05 - acc: 1.0000 - val_loss: 0.1619 - val_acc: 0.9567
Epoch 64/100
4800/4800 [==============================] - 30s 6ms/step - loss: 2.9331e-05 - acc: 1.0000 - val_loss: 0.1590 - val_acc: 0.9575
Epoch 65/100
4800/4800 [==============================] - 30s 6ms/step - loss: 3.4036e-05 - acc: 1.0000 - val_loss: 0.1558 - val_acc: 0.9558
Epoch 66/100
4800/4800 [==============================] - 30s 6ms/step - loss: 2.9944e-05 - acc: 1.0000 - val_loss: 0.1577 - val_acc: 0.9550
Epoch 67/100
4800/4800 [==

In [16]:
from tensorflow.keras.models import load_model
best_model = load_model('models/DenseNet121_normalization.h5')

In [18]:
evaluate = best_model.evaluate(valid_x, valid_y)
evaluate

48/48 [==============================] - 7s 145ms/step


[0.6185124516487122, 0.8958333333333334]

In [15]:
#submission = pd.read_csv('data/val.csv')
#submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
#submission.to_csv('data/val(CNN).csv', index=False)